In [18]:
import ciao_contrib.runtool as rt
import sherpa.astro.ui as ui
import numpy as np
import matplotlib.pyplot as plt
import os
import re

#astropy is used to read WCS position information from fits files for region_count
from astropy.io import fits
from astropy import wcs
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance


def region_count(obsid_in):
    '''
    return the count rate for an obsid with upper and lower bounds on the 90% confidence interval
    '''
    srcflx = rt.srcflux
    srcflx.punlearn() #Restore the system defaults for a CIAO parameter file.
    srcflx.infile = f'data/{obsid_in}/repro/acisf{obsid_in}_repro_evt2.fits'
    with fits.open(srcflx.infile) as f:
        loc = SkyCoord(f[1].header['RA_NOM']*u.deg,f[1].header['DEC_NOM']*u.deg)
        loc_str = loc.ra.to_string(unit=u.hourangle, sep=':', pad=True, precision=2) + " " + loc.dec.to_string(unit=u.degree, sep=':', pad=True, precision=2)
    srcflx.pos = loc_str #Reads RA, DEC columns as a position in sexagesimal "degree:arcmin:arcsec" format
    srcflx.outroot = f'data/{obsid_in}/repro/srcflux_products/'
    srcflx.srcreg = f'regions/srcwcs.reg' #region used to calculate the source counts
    srcflx.bkgreg = f'regions/bkgwcs.reg'#region used to calculate the background counts
    srcflx.clobber = 'yes'
    srcflx.verbose = 0 #Default =1, 0 suppresses the long print statement

    #a _summary.txt file is produces with the count rate and an unjustified flux
    with open(srcflx.outroot+'_summary.txt', encoding='utf-8') as f:
        for line in f:
            count_rate_match = re.search('0\\.0.*(c/s)',line)
            lb_match = re.search('\\(0\\.0.*\\,', line)
            ub_match = re.search('\\,0\\.0.*\\)', line)
            #0001|13 9 48.47 -23 22 58.4  Rate           0.0014 c/s (0.00104,0.00183)      
            if count_rate_match:
                cr = float(count_rate_match.group(0)[0:-4])
                lb = float(lb_match.group(0)[1:-1])
                ub = float(ub_match.group(0)[1:-1])
                return(cr, lb, ub)


def merge_observations(obsid_in_list):
    '''
    params
    ------
    obsid_in_list : list
        List of observation IDs to be merged

    returns
    -------
    None
    
    outputs
    -------
    produces img and fov files
    '''

    flxobs = rt.flux_obs
    flxobs.punlearn()
    flxobs.infiles = str()
    for obsid in obsid_in_list:
        if flxobs.infiles is None:
            flxobs.infiles = f'data/{obsid}/repro/acisf{obsid}_repro_evt2.fits'
        else:
            flxobs.infiles += f',data/{obsid}/repro/acisf{obsid}_repro_evt2.fits'
    print(flxobs.infiles)
    flxobs.outroot = 'data/merge_test/'
    #flxobs()

def merge_by_epoch(obsid_in_list):
    '''
    params
    ------
    obsid_in_list : list
        List of observation IDs to be merged

    returns
    -------
    None
    '''
    mrgobs = rt.merge_obs
    mrgobs.punlearn()
    mrgobs.infiles = str()
    for obsid in obsid_in_list:
        if mrgobs.infiles is None:
            mrgobs.infiles = f'data/{obsid}/repro/acisf{obsid}_repro_evt2.fits'
        else:
            mrgobs.infiles += f', data/{obsid}/repro/acisf{obsid}_repro_evt2.fits' ## includes leading comma
    print(mrgobs.infiles)
    mrgobs.outroot = 'data/merge_test/'

import spreadsheet.ipynb

def lightcurve_counts(obsid_in_list):
    '''
    produce a light curve of count rate vs time
    '''
    counts_list = []
    t_list = []
    for obsid in obsid_in_list:
        [count, lb, ub] = region_count(obsid)
        counts_list.append(count)
        [t_t0,exp_time]=spreadsheet.obsid_header_parse(obsid)
        t_list.append(t_t0)
    
    fig = plt.figure()
    plt.errorbar(t_list, counts_list, fmt='bo', yerr=lb+ub, xerr=exp_time)    

ModuleNotFoundError: No module named 'spreadsheet'

In [17]:
#reproject(20936, 20860)
obsid = 20861
[count, lb, ub] = region_count(obsid)
print(f'Obs ID {obsid} observed a count rate of {count} c/s')

from spreadsheet.ipynb import epoch_obsid_list as eol
lightcurve_counts(eol)

#merge_observations([20860, 20936])

Obs ID 20861 observed a count rate of 0.0014 c/s
